In [ ]:

!pip install pinecone-client==3.1.0
!pip install pinecone-datasets==0.7.0
!pip install  sentence-transformers==2.2.2


In [1]:
apikey='API KEY'

In [88]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
import torch
from pinecone_datasets import load_dataset
from sentence_transformers import SentenceTransformer
import pandas as pd

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [19]:
pc = Pinecone(api_key=apikey)# se instancia el objeto

In [18]:
specs = ServerlessSpec(cloud='aws', region='us-east-1') # se configura la infraestructura

In [69]:


dataset = load_dataset('quora_all-MiniLM-L6-bm25') # se carga un dataset de ejemplo

In [70]:
del dataset.documents['metadata']

In [71]:
dataset.documents.rename(columns={'blob':'metadata'},inplace=True)

In [72]:
dataset.documents.loc[0,'values'].shape # esta es la lonogitud de los embeddings para ese dataset

(384,)

In [73]:
dataset.documents.drop(dataset.documents.index[1000:], inplace=True) # dejamos solo 1000 filas por practicidad

In [74]:
dataset.documents

,id,values,sparse_values,metadata
0,1,"[0.06814987, -0.039664183, -0.06096721, 0.0074...","{'indices': [7096, 8508, 13677, 23041, 24734, ...",{'text': ' What is the step by step guide to i...
1,2,"[0.08983771, -0.03493085, -0.057357617, 0.0222...","{'indices': [7096, 8508, 13677, 24734, 26026, ...",{'text': ' What is the step by step guide to i...
2,3,"[-0.046798065, 0.1551149, -0.03920019, 0.04878...","{'indices': [6065, 13677, 17109, 20780, 24734,...",{'text': ' What is the story of Kohinoor (Koh-...
3,4,"[-0.077349104, 0.14786911, -0.0128817065, -0.0...","{'indices': [2408, 6065, 7582, 12225, 17109, 2...",{'text': ' What would happen if the Indian gov...
4,5,"[-0.028324936, 0.037209604, -0.00040033547, 0....","{'indices': [5388, 12812, 18181, 19960, 20780,...",{'text': ' How can I increase the speed of my ...
...,...,...,...,...
995,1027,"[0.012958857, -0.019112129, -0.0357613, 0.1456...","{'indices': [19960, 23463, 26754, 26853, 39832...",{'text': ' Does anemia increase irritability?'}
996,1028,"[-0.000101657635, 0.03107518, 0.07726865, -0.0...","{'indices': [11129, 11710, 21006, 23100, 24734...",{'text': ' How do you feel about the new MacBo...
997,1029,"[-0.00540815, -0.04188697, 0.06312727, -0.0552...","{'indices': [11710, 13730, 21006, 24734, 25991...",{'text': ' Should I get the new Macbook Pro?'}
998,1030,"[0.034669217, 0.07669944, 0.01589522, 0.047766...","{'indices': [10311, 14506, 15984, 23463, 38585...",{'text': ' Does covering your head while sleep...


In [63]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2',device=device) # tambien codifica embeddings de longitud 384
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [76]:
 pc.list_indexes() # revisar si tenemos indexes

{'indexes': []}

In [75]:
pc.delete_index("busqueda-andres") # en caso de borrar alguno

In [77]:
index_title='busqueda-prueba'# nuevo indice

In [78]:
pc.create_index(
        index_title,
        dimension=384,
        metric='dotproduct',
        spec=specs
    )

In [79]:
index = pc.Index(index_title)

In [80]:
from tqdm import tqdm

for batch in tqdm(dataset.iter_documents(batch_size=100)): # se inserta la información
    index.upsert(batch)

10it [00:04,  2.27it/s]


In [92]:
# se generaron algunas palabras que actuarán como querys
tech_words = [
    "algorithm", "API", "application", "architecture", "artificial intelligence", "augmented reality",
    "automation", "bandwidth", "big data", "binary", "bitcoin", "blockchain", "bot", "browser",
    "byte", "cache", "cloud", "coding", "compiler", "compression", "computing", "connectivity",
    "crypto", "cybersecurity", "database", "data mining", "data science", "debugging", "deployment",
    "digital", "DNS", "encryption", "enterprise", "firewall", "firmware", "framework", "frontend",
    "full stack", "GPU", "hardware", "HTML", "HTTP", "HTTPS", "hyperlink", "IoT", "IP address",
    "Java", "JavaScript", "kernel", "latency", "load balancing", "machine learning", "malware",
    "microservices", "middleware", "mobile", "module", "motherboard", "multithreading", "network",
    "node", "object-oriented", "open source", "operating system", "packet", "password", "payload",
    "ping", "pixel", "platform", "plugin", "port", "protocol", "Python", "QR code", "query",
    "RAM", "reactive programming", "REST", "router", "runtime", "SaaS", "scalability", "SDK",
    "server", "shell", "SMTP", "software", "SQL", "SSL", "stack", "storage", "streaming",
    "syntax", "TCP/IP", "telecommunication", "token", "UI", "URL", "user experience", "UX",
    "virtualization", "VPN", "VR", "web development", "Wi-Fi", "XML", "zip file", "zero-day",
    "3D printing", "access control", "adblock", "agile", "analytics", "android", "API gateway",
    "app store", "application server", "artificial neural network", "asset management", "attack vector",
    "authentication", "authorization", "back end", "backup", "bandwidth throttling", "behavioral biometrics",
    "bioinformatics", "biometrics", "bitrate", "blockchain", "blueprint", "Bluetooth", "bootloader",
    "botnet", "broadband", "broadcast", "browser extension", "bus", "bytecode", "cache memory",
    "captcha", "CDN", "cellular network", "chatbot", "chip", "CIO", "cipher", "cloud computing",
    "cloud storage", "cluster", "codec", "code review", "command line", "compilation", "component",
    "computational linguistics", "computer graphics", "computer vision", "content management system",
    "content delivery network", "continuous integration", "control panel", "cookies", "cryptanalysis",
    "cryptography", "CSS", "cyber attack", "cyber defense", "cyber hygiene", "cyber incident",
    "cyber insurance", "cyber law", "cyber operation", "cyber physical system", "cyber resilience",
    "cyber threat", "cyber warfare", "data breach", "data center", "data encryption", "data integrity",
    "data loss prevention", "data privacy", "data protection", "data recovery", "data visualization",
    "deep learning", "DevOps", "digital currency", "digital footprint", "digital identity",
    "digital marketing", "digital rights"]


In [93]:
# se hacen las consultas y se toman los 3 valores mas cercanos
final_results={}
for query in tqdm(tech_words):
  embedded = model.encode(query).tolist()
  output = index.query(vector=embedded, top_k=3, include_metadata=True)
  final_results[query]=output['matches']


100%|██████████| 194/194 [00:06<00:00, 32.08it/s]


In [95]:
final_results['computer graphics']

[{'id': '866',
  'metadata': {'text': ' What is the best forum for graphic designers?'},
  'score': 0.429079294,
  'values': []},
 {'id': '867',
  'metadata': {'text': ' Who is the best graphic designer in israel?'},
  'score': 0.384842515,
  'values': []},
 {'id': '237',
  'metadata': {'text': ' What are some mind-blowing computer tools that exist '
                       "that most people don't know about?"},
  'score': 0.361391336,
  'values': []}]

In [97]:
final_results['3D printing']

[{'id': '191',
  'metadata': {'text': ' How does 3D printing work?'},
  'score': 0.87873584,
  'values': []},
 {'id': '192',
  'metadata': {'text': ' How do 3D printing work?'},
  'score': 0.877525091,
  'values': []},
 {'id': '930',
  'metadata': {'text': ' Considering the current trend of information '
                       'technology what is the future of print media?'},
  'score': 0.396922082,
  'values': []}]